In [21]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/home/jvalenti/MOAD/analysis-jose/Source') #Dir where OP_kernels and OP_funtions are located.
from OP_Kernels import DeleteParticle
from OP_functions import *

In [22]:
# Define paths (This function you would need to change with your own paths)
local = 0 #Set to 0 when working on server
paths = path(local) 

In [23]:
def Beaching(particle, fieldset, time):
    '''Beaching prob'''  
    if particle.beached == 0: #Check particle is in the water column       
        Tb = particle.Lb*86400 #timescale beaching in seconds
        x_offset = particle.Db/(111319.5*cos(particle.lat*(math.pi/180))) #Checking distance x of possible beaching
        y_offset = particle.Db/111319.5 #Checking distance y of possible beaching
        Pb = 1 - exp(-particle.dt/Tb)
        if particle.lat < 48.6 and particle.lon < -124.7 or particle.lat < 49.237 and particle.lon > -123.196 and particle.lat > 49.074:
            pass #Dont let particles beach inside the fraser river
        elif ParcelsRandom.uniform(0,1)<Pb:
            DWS1 = fieldset.U[time, 0.5, particle.lat+y_offset, particle.lon+x_offset] #particle.depth 0.5 check surface beach
            DWS2 = fieldset.U[time, 0.5, particle.lat-y_offset, particle.lon+x_offset]  
            DWS3 = fieldset.U[time, 0.5, particle.lat-y_offset, particle.lon-x_offset]
            DWS4 = fieldset.U[time, 0.5, particle.lat+y_offset, particle.lon-x_offset]
            if DWS1 == 0 or DWS2 == 0 or DWS3 == 0 or DWS4 == 0:
                particle.beached = 1

def AdvectionRK4_3D(particle, fieldset, time):
    if particle.beached == 0: #Check particle is in the water column
        (u1, v1, w1) = fieldset.UVW[time, particle.depth, particle.lat, particle.lon]
        lon1 = particle.lon + u1*.5*particle.dt
        lat1 = particle.lat + v1*.5*particle.dt
        dep1 = particle.depth + w1*.5*particle.dt
        (u2, v2, w2) = fieldset.UVW[time + .5 * particle.dt, dep1, lat1, lon1]
        lon2 = particle.lon + u2*.5*particle.dt
        lat2 = particle.lat + v2*.5*particle.dt
        dep2 = particle.depth + w2*.5*particle.dt
        (u3, v3, w3) = fieldset.UVW[time + .5 * particle.dt, dep2, lat2, lon2]
        lon3 = particle.lon + u3*particle.dt
        lat3 = particle.lat + v3*particle.dt
        dep3 = particle.depth + w3*particle.dt
        (u4, v4, w4) = fieldset.UVW[time + particle.dt, dep3, lat3, lon3]
        particle.lon += (u1 + 2*u2 + 2*u3 + u4) / 6. * particle.dt
        particle.lat += (v1 + 2*v2 + 2*v3 + v4) / 6. * particle.dt
        particle.depth += (w1 + 2*w2 + 2*w3 + w4) / 6. * particle.dt

## Definitions

In [24]:
start = datetime(2020, 1, 1) #Start date
# Set Time length [days] and timestep [seconds]
length = 1 #days
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 1 # number of deploying locations
n = 1 # 1000   # number of particles per location
dmin = 10 #minimum depth
dd = 5 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [25]:
from parcels import Variable

class MPParticle(JITParticle):        
    Lb = Variable('Lb', initial = 0.23)  #days needed in days for particle to have 67% probability of beaching if in beaching zone (500m)
    Db = Variable('Db', initial = 33) #Distance at which particles can randomly beach.
    beached = Variable('beached', initial = 0)

In [26]:
clon = [-123.391892] #list of your longitudes in same order you give the latitudes 
clat = [49.195884]
lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [27]:
#Set start date time and the name of the output file
name = 'test_beaching' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)

In [28]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

In [29]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, repeatdt = timedelta(hours=1))


k_beach = pset.Kernel(Beaching)

pset.execute(AdvectionRK4_3D + k_beach,
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBeaching ==> /tmp/parcels-2894/lib689bfb679329018d838df8af82399fe1_0.so
INFO: Temporary output files are stored in /home/jvalenti/MOAD/results/out-CYCVIOGM.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/results/out-CYCVIOGM" to convert these to a NetCDF file during the run.
N/A% (0 of 86400.0) |                    | Elapsed Time: 0:00:00 ETA:  --:--:--
 16% (14400.0 of 86400.0) |##            | Elapsed Time: 0:00:02 ETA:   0:00:10
 20% (18000.0 of 86400.0) |##            | Elapsed Time: 0:00:04 ETA:   0:00:39
 25% (21600.0 of 86400.0) |###           | Elapsed Time: 0:00:06 ETA:   0:00:36
 29% (25200.0 of 86400.0) |####          | Elapsed Time: 0:00:08 ETA:   0:00:36
 33% (28800.0 of 86400.0) |####          | Elapsed Time: 0:00:10 ETA:   0:00:32
 37% (32400.0 of 86400.0) |#####         | Elapsed Time: 0:00:12 ETA:   0:00:30
 41% (36000.0 of 86400.0) |#####         | Elapsed Time: 0:00:14 ETA:   0:00:29
 45% (39600.0 